# Passing through instance keys and features when using a keras model

This notebook will show you how to modify a Keras model, either loaded from disk or newly created, to perform keyed predictions or forward features through with the prediction.

## Topics Covered
- Modify serving signature of existing model to accept and forward keys
- Multiple serving signatures on one model
- Online and batch predictions with Google Cloud AI Platform
- Forward features in model definition
- Forward features with serving signature

#TODO:
 - loaded model or not loaded model
 - Model.predict() doesn't take signature
 - parameterize the bash cells a little more
 - add more conceptual explanation when finish out the blog itself
 - add link to actual blog
 
Q's
 - worth including BQML?
 - model.predict() doesn't seem to respect Input() layer name?
 - model.predict() doesn't output keys for the prediction dictionary
 - feature forward model has both signatures, too much?
 - tool long in general?
 - should load a saved model instead? I sort of liked that they saw where the layers were named and then corresponding key in teh signature
 - model in a model still 'feels' cleaner even though not recommended? don't have to mess with tf.function

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.1.0-dlenv_tfe'

## Build and Train a Fashion MNIST model

We will use a straightforward keras use case with the fashion mnist dataset to demonstrate building a model and then adding support for keyed predictions.
More here:
https://colab.sandbox.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/classification.ipynb

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [4]:
# Scale down dataset
train_images = train_images / 255.0
test_images = test_images / 255.0

In [5]:
# Build and traing model

from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Flatten

model = Sequential([
  Input(shape=(28,28), name="image"),
  Flatten(input_shape=(28, 28), name="flatten"),
  Dense(64, activation='relu', name="dense"),
  Dense(10, activation='softmax', name="preds"),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Only training for 1 epoch, we are not worried about model performance
model.fit(train_images, train_labels, epochs=1, batch_size=32)

Train on 60000 samples
60000/60000 [==============================] - 12s 194us/sample - loss: 0.5175 - accuracy: 0.8210


In [6]:
# Create test_image
test_image = np.expand_dims(test_images[0],0).astype('float32')
model.predict(test_image)

array([[3.4721153e-05, 2.2903264e-06, 9.4477455e-06, 5.1241836e-06,
        6.8963200e-05, 1.8251964e-01, 5.0299277e-05, 6.8799429e-02,
        1.5516685e-03, 7.4695832e-01]], dtype=float32)

## SavedModel and serving signature

Now save the model using tf.saved_model.save(). This will add a serving signature which we can then inspect. The serving signature indicates exactly which input names and types are expected, and what will be output by the model

In [7]:
MODEL_EXPORT_PATH = './model/'
tf.saved_model.save(model, MODEL_EXPORT_PATH)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./model/assets


In [8]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {MODEL_EXPORT_PATH}

The given SavedModel SignatureDef contains the following input(s):
  inputs['image'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_image:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['preds'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [9]:
# Load the model from storage and inspect the object types
loaded_model = tf.keras.models.load_model(MODEL_EXPORT_PATH)
loaded_model.signatures

_SignatureMap({'serving_default': <tensorflow.python.saved_model.load._WrapperFunction object at 0x7f9d9c0dba50>})

In [10]:
loaded_model

It's worth noting that original model did not have serving signature until we saved it and is a slightly different object type:

In [11]:
model

In [12]:
# Expect an Error
model.signatures

AttributeError: 'Sequential' object has no attribute 'signatures'

## Standard serving function

We can actually get access to the inference_function of the loaded model and is it directly to perform predictions, similar to a Keras Model.predict() call. Note the name of the output Tensor matches the serving signature.

In [13]:
inference_function = loaded_model.signatures['serving_default']

print(inference_function)

In [14]:
result = inference_function(tf.constant(test_image))

print(result)

{'preds': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[3.4721153e-05, 2.2903264e-06, 9.4477455e-06, 5.1241836e-06,
        6.8963200e-05, 1.8251964e-01, 5.0299277e-05, 6.8799429e-02,
        1.5516685e-03, 7.4695832e-01]], dtype=float32)>}


In [15]:
# Matches serving signature
result['preds']

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[3.4721153e-05, 2.2903264e-06, 9.4477455e-06, 5.1241836e-06,
        6.8963200e-05, 1.8251964e-01, 5.0299277e-05, 6.8799429e-02,
        1.5516685e-03, 7.4695832e-01]], dtype=float32)>

## Keyed Serving Function

Now we'll create a new serving function that accepts and outputs a unique instance key. We use the fact that a Keras Model(x) call actually runs a prediction. The training=False parameter is included only for clarity. Then we save the model as before but provide this function as our new serving signature.

In [16]:
@tf.function(input_signature=[tf.TensorSpec([None], dtype=tf.string),tf.TensorSpec([None, 28, 28], dtype=tf.float32)])
def keyed_prediction(key, image):
    pred = loaded_model(image, training=False)
    return {
        'preds': pred,
        'key': key
    }

In [17]:
# Resave model, but specify new serving signature
KEYED_EXPORT_PATH = './keyed_model/'
loaded_model.save(KEYED_EXPORT_PATH, signatures={'serving_default': keyed_prediction})

INFO:tensorflow:Assets written to: ./keyed_model/assets


In [18]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {KEYED_EXPORT_PATH}

The given SavedModel SignatureDef contains the following input(s):
  inputs['image'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_image:0
  inputs['key'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_key:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['key'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['preds'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


In [19]:
keyed_model = tf.keras.models.load_model(KEYED_EXPORT_PATH)

In [20]:
# TODO: why does this not work?
# It won't even accept the Input layer name, and instead requires the autogenerated name of the 'flatten' layer
keyed_model.predict({
    'flatten_input': test_image,
    'key': tf.constant("unique_key")}
)
# keyed_model.predict(test_image)

array([[3.4721153e-05, 2.2903264e-06, 9.4477455e-06, 5.1241836e-06,
        6.8963200e-05, 1.8251964e-01, 5.0299277e-05, 6.8799429e-02,
        1.5516685e-03, 7.4695832e-01]], dtype=float32)

## Multiple Signature Model

Sometimes it is useful to leave both signatures in the model definition so the user can indicate if they are performing a keyed prediction or not. This can easily be done with the model.save() method as before.

In general, your serving infrastructure will default to 'serving_default' unless otherwise specified in a prediction call. Google Cloud AI Platform online and batch prediction support multiple signatures, as does [TFServing](https://www.tensorflow.org/tfx/serving/api_rest#request_format_2).

In [21]:
# Using inference_function from earlier
DUAL_SIGNATURE_EXPORT_PATH = './dual_signature_model/'
loaded_model.save(DUAL_SIGNATURE_EXPORT_PATH, signatures={'serving_default': keyed_prediction,
                                                  'unkeyed_signature': inference_function})

INFO:tensorflow:Assets written to: ./dual_signature_model/assets


In [22]:
# Examine the multiple signatures
!saved_model_cli show --tag_set serve --dir {KEYED_EXPORT_PATH}

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [23]:
# Default signature
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {KEYED_EXPORT_PATH}

The given SavedModel SignatureDef contains the following input(s):
  inputs['image'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_image:0
  inputs['key'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_key:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['key'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['preds'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


In [24]:
# Alternative unkeyed signature
!saved_model_cli show --tag_set serve --signature_def unkeyed_signature --dir {KEYED_EXPORT_PATH}

The given SavedModel SignatureDef contains the following input(s):
The given SavedModel SignatureDef contains the following output(s):
Method name is: 


## Deploy the model and perform predictions

Now we'll deploy the model to AI Platform serving and perform both online and batch keyed predictions. Deployment will take 2-3 minutes.

In [25]:
%%bash

MODEL_LOCATION='./dual_signature_model/'
MODEL_NAME=fashion_mnist
MODEL_VERSION=v1

TFVERSION=2.1
REGION=us-central1
BUCKET=dhodun1

# create the model if it doesn't already exist
modelname=$(gcloud ai-platform models list | grep -w "$MODEL_NAME")
echo $modelname
if [ -z "$modelname" ]; then
   echo "Creating model $MODEL_NAME"
   gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
else
   echo "Model $MODEL_NAME already exists"
fi

# delete the model version if it already exists
modelver=$(gcloud ai-platform versions list --model "$MODEL_NAME" | grep -w "$MODEL_VERSION")
echo $modelver
if [ "$modelver" ]; then
   echo "Deleting version $MODEL_VERSION"
   yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
   sleep 10
fi


echo "Creating version $MODEL_VERSION from $MODEL_LOCATION"
gcloud ai-platform versions create ${MODEL_VERSION} \
       --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --staging-bucket gs://${BUCKET} \
       --runtime-version $TFVERSION

fashion_mnist v1
Model fashion_mnist already exists
v1 gs://dhodun1/c89caeaabecb8a1820b7163a264899db920fceab2404bd34ea6f78458eb44ffa/ READY
Deleting version v1
Creating version v1 from ./dual_signature_model/


This will delete version [v1]...

Do you want to continue (Y/n)?  
Deleting version [v1]......
..............................................................................................................................................................done.
Creating version (this might take a few minutes)......
...........................................................................................................................................................................................................................................................................................................................done.


In [26]:
# TODO: There a way to paramterize %%writefile?
# Create keyed test_image file

with open("keyed_input.json", "w") as file:
    print(f'{{"image": {test_image.tolist()}, "key": "hi"}}', file=file)

In [27]:
# Single online keyed prediction, --signature-name is not required since we're hitting the default but shown for clarity

!gcloud ai-platform predict --model fashion_mnist --json-instances keyed_input.json --version v1 --signature-name serving_default

KEY  PREDS
hi   [3.472115349723026e-05, 2.2903222998138517e-06, 9.447747288504615e-06, 5.124184099258855e-06, 6.896320701343939e-05, 0.1825195848941803, 5.029932799516246e-05, 0.0687994509935379, 0.0015516685089096427, 0.7469583749771118]


In [28]:
# Create unkeyed test_image file

with open("unkeyed_input.json", "w") as file:
    print(f'{{"image": {test_image.tolist()}}}', file=file)

In [29]:
# Single online unkeyed prediction using alternative serving signature

!gcloud ai-platform predict --model fashion_mnist --json-instances unkeyed_input.json --version v1 --signature-name unkeyed_signature

PREDS
[3.472115349723026e-05, 2.2903222998138517e-06, 9.447747288504615e-06, 5.124184099258855e-06, 6.896320701343939e-05, 0.1825195848941803, 5.029932799516246e-05, 0.0687994509935379, 0.0015516685089096427, 0.7469583749771118]


## Batch Predictions

Now we'll create multiple keyed prediction files and create a job to perform these predictions in a scalable, distributed manner. The keys will be retained so the results can be stored and associated with the initial inputs.

In [30]:
# Create Data files:
import os
import shutil

DATA_DIR = './batch_data'
shutil.rmtree(DATA_DIR, ignore_errors=True)
os.makedirs(DATA_DIR)

# Create 10 files with 10 images each
for i in range(10):
    with open(f'{DATA_DIR}/keyed_batch_{i}.json', "w") as file:
        for z in range(10):
            key = f'key_{i}_{z}'
            print(f'{{"image": {test_images[z].tolist()}, "key": "{key}"}}', file=file)

In [31]:
%%bash
BUCKET=dhodun1
gsutil -m cp -r ./batch_data gs://$BUCKET/temp/

Copying file://./batch_data/keyed_batch_1.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_9.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_8.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_3.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_5.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_6.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_7.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_2.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_4.json [Content-Type=application/json]...
Copying file://./batch_data/keyed_batch_0.json [Content-Type=application/json]...
/ [10/10 files][870.0 KiB/870.0 KiB] 100% Done                                  
Operation completed over 10 objects/870.0 KiB.                                   


This following batch prediction job took me 8-10 minutes, most of the time spent in infrastructure spin up.

In [32]:
%%bash

DATA_FORMAT="text" # JSON data format
INPUT_PATHS='gs://dhodun1/temp/batch_data/*'
OUTPUT_PATH='gs://dhodun1/temp/batch_predictions'
MODEL_NAME='fashion_mnist'
VERSION_NAME='v1'
REGION='us-central1'
now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="fashion_mnist_batch_predict_$now"
LABELS="team=engineering,phase=test,owner=drew"
SIGNATURE_NAME="serving_default"

gcloud ai-platform jobs submit prediction $JOB_NAME \
    --model $MODEL_NAME \
    --version $VERSION_NAME \
    --input-paths $INPUT_PATHS \
    --output-path $OUTPUT_PATH \
    --region $REGION \
    --data-format $DATA_FORMAT \
    --labels $LABELS \
    --signature-name $SIGNATURE_NAME

jobId: fashion_mnist_batch_predict_20200611_162203
state: QUEUED


Job [fashion_mnist_batch_predict_20200611_162203] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe fashion_mnist_batch_predict_20200611_162203

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs fashion_mnist_batch_predict_20200611_162203


In [50]:
# You can stream the logs, this cell will block until the job completes.
# Copy and paste from the previous cell's output based to grab your job name

# gcloud ai-platform jobs stream-logs fashion_mnist_batch_predict_20200611_151356

In [34]:
!gsutil ls gs://dhodun1/temp/batch_predictions

gs://dhodun1/temp/batch_predictions/prediction.errors_stats-00000-of-00001
gs://dhodun1/temp/batch_predictions/prediction.results-00000-of-00001
gs://dhodun1/temp/batch_predictions/prediction.results-00000-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00001-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00002-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00003-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00004-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00005-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00006-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00007-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00008-of-00010
gs://dhodun1/temp/batch_predictions/prediction.results-00009-of-00010


In [35]:
# View predictions with keys
!gsutil cat 'gs://dhodun1/temp/batch_predictions/prediction.results-00000-of-00010'

{"preds": [4.013196667074226e-05, 7.403023118968122e-06, 3.707501718963613e-06, 1.3309241921888315e-06, 2.7610798497335054e-05, 0.12554031610488892, 4.4301763409748673e-05, 0.2407132089138031, 0.005084679462015629, 0.6285373568534851], "key": "key_8_0"}
{"preds": [0.0017545941518619657, 2.4121516162267653e-06, 0.8372505307197571, 0.0012869687052443624, 0.0498567670583725, 1.833312808230403e-07, 0.10880657285451889, 2.6277988118827977e-10, 0.0010421136394143105, 5.360926813580136e-09], "key": "key_8_1"}
{"preds": [2.7995271011604927e-05, 0.9999473094940186, 1.4881048855386325e-06, 1.5070620065671392e-05, 7.655904482817277e-06, 4.014826515685854e-08, 1.3820189792568272e-07, 1.0850141229923338e-08, 1.973001246824424e-07, 4.457038116356671e-08], "key": "key_8_2"}
{"preds": [1.7342121282126755e-05, 0.9997573494911194, 1.1361087672412395e-05, 0.00018262902449350804, 2.446685357426759e-05, 3.6486651424638694e-06, 1.1849277825604076e-06, 1.1300596014507391e-07, 1.1253807770117419e-06, 7.832731

## BigQuery ML Batch Predictions


#TODO: change shape of model so it works? Doesn't support (28,28 input shape)

If your source data exists in BigQuery, an alternative is to load your Tensorflow model into [BQML](https://cloud.google.com/bigquery-ml/docs/making-predictions-with-imported-tensorflow-models#overview) and perform predictions directly in the data warehouse. This also removes the need to produced keyed predictions at all.

[Not all datatypes are supported](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-tensorflow#inputs) and some use cases are less conducive to this, in our case, an image model.

Loading the model would look something like this:

```
CREATE OR REPLACE MODEL my_dataset.fashion_mnist
OPTIONS (MODEL_TYPE='TENSORFLOW',
    MODEL_PATH='gs://dhodun1/temp/model/*')
```

And performing batch predictions:

#TODO: are other features forwarded along?

```
SELECT
  *
FROM
  ML.PREDICT(MODEL `my_dataset.fashion_mnist`,
    (
    SELECT
      image,
      
    FROM
      `my_datset.images`))
```


## Feature Forward Models

There are also times where it's desirable to forward some or all of the input features along with the output. This can be achieved in a very similar manner as adding keyed outputs to our model.

Note that this will be a little trickier to grab a subset of features if you are feeding all of your input features as a single Input() layer in the Keras model. This example takes multiple Inputs.

In [36]:
# Build a toy model using the Boston Housing dataset
# https://www.kaggle.com/c/boston-housing
# Prediction target is median value of homes in $1000's

(train_data, train_targets), _ = keras.datasets.boston_housing.load_data()

# Extract just two of the features for simplicity's sake
train_tax_rate = train_data[:,10]
train_rooms = train_data[:,5]

In [37]:
# Build a toy model with multiple inputs
# This time using the Keras functional API

from tensorflow.keras.layers import Input
from tensorflow.keras import Model


tax_rate = Input(shape=(1,), dtype=tf.float32, name="tax_rate")
rooms = Input(shape=(1,), dtype=tf.float32, name="rooms")

x = tf.keras.layers.Concatenate()([tax_rate, rooms])
x = tf.keras.layers.Dense(64, activation='relu')(x)
price = tf.keras.layers.Dense(1, activation=None, name="price")(x)

# Functional API model instead of Sequential
model = Model(inputs=[tax_rate, rooms], outputs=[price])

In [38]:
model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['accuracy']
    )
model.fit([train_tax_rate, train_rooms], train_targets, epochs=10)

Train on 404 samples
Epoch 1/10
404/404 [==============================] - 0s 1ms/sample - loss: 537.3960 - accuracy: 0.0000e+00
Epoch 2/10
404/404 [==============================] - 0s 96us/sample - loss: 418.1529 - accuracy: 0.0000e+00
Epoch 3/10
404/404 [==============================] - 0s 94us/sample - loss: 320.9664 - accuracy: 0.0000e+00
Epoch 4/10
404/404 [==============================] - 0s 103us/sample - loss: 246.3120 - accuracy: 0.0000e+00
Epoch 5/10
404/404 [==============================] - 0s 90us/sample - loss: 190.5645 - accuracy: 0.0000e+00
Epoch 6/10
404/404 [==============================] - 0s 112us/sample - loss: 149.0178 - accuracy: 0.0000e+00
Epoch 7/10
404/404 [==============================] - 0s 104us/sample - loss: 125.7651 - accuracy: 0.0000e+00
Epoch 8/10
404/404 [==============================] - 0s 121us/sample - loss: 109.7265 - accuracy: 0.0000e+00
Epoch 9/10
404/404 [==============================] - 0s 100us/sample - loss: 102.1707 - accuracy: 0.000

## Feature forward and non feature forward predictions

Using the Keras sequential API, we create another model with slightly different inputs and outputs, but retaining the weights of the existing model. Notice the predictions with and without feature forwarding.

In [39]:
model.predict({
    'tax_rate': tf.convert_to_tensor([20.2]),
    'rooms': tf.convert_to_tensor([6.2])
})

array([[22.480536]], dtype=float32)

In [40]:
BOSTON_EXPORT_PATH = './boston_model/'
model.save(BOSTON_EXPORT_PATH)

INFO:tensorflow:Assets written to: ./boston_model/assets


In [41]:
# Will retain weights from trained model but also forward out a feature
forward_model = Model(inputs=[tax_rate, rooms], outputs=[price, tax_rate])

In [42]:
# Notice we get both outputs now
# TODO: why lack of keys on the prediction dictionary?
forward_model.predict({
    'tax_rate': tf.convert_to_tensor([5.0]),
    'rooms': tf.convert_to_tensor([6.2])
})

[array([[9.465022]], dtype=float32), array([[5.]], dtype=float32)]

In [43]:
FORWARD_EXPORT_PATH = './forward_model/'
forward_model.save(FORWARD_EXPORT_PATH)

INFO:tensorflow:Assets written to: ./forward_model/assets


In [44]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {FORWARD_EXPORT_PATH}

The given SavedModel SignatureDef contains the following input(s):
  inputs['rooms'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_rooms:0
  inputs['tax_rate'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_tax_rate:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['price'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
  outputs['tax_rate'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


## Forwarding by changing serving signature

We could have employed the same method as before to also modify the serving signature and save out the model to achieve the same result.

In [45]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {BOSTON_EXPORT_PATH}

The given SavedModel SignatureDef contains the following input(s):
  inputs['rooms'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_rooms:0
  inputs['tax_rate'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_tax_rate:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['price'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [46]:
#TODO: Is there way to avoid having to create this? In the previous example we grabbed the funcion from a Loaded model
@tf.function(input_signature=[tf.TensorSpec([None, 1], dtype=tf.float32), tf.TensorSpec([None, 1], dtype=tf.float32)])
def standard_forward_prediction(tax_rate, rooms):
    pred = model([tax_rate, rooms], training=False)
    return {
        'price': pred,
    }

In [47]:
# Return out the feature of interest as well as the prediction
@tf.function(input_signature=[tf.TensorSpec([None, 1], dtype=tf.float32), tf.TensorSpec([None, 1], dtype=tf.float32)])
def feature_forward_prediction(tax_rate, rooms):
    pred = model([tax_rate, rooms], training=False)
    return {
        'price': pred,
        'tax_rate': tax_rate
    }

In [48]:
# Save out the model with both signatures
DUAL_SIGNATURE_FORWARD_PATH = './dual_signature_forward_model/'
model.save(DUAL_SIGNATURE_FORWARD_PATH, signatures={'serving_default': standard_forward_prediction,
                                   'feature_forward': feature_forward_prediction})

INFO:tensorflow:Assets written to: ./dual_signature_forward_model/assets


In [49]:
# Inspect just the feature_forward signature, but we also have standard serving_default
!saved_model_cli show --tag_set serve --signature_def feature_forward --dir {DUAL_SIGNATURE_FORWARD_PATH}

The given SavedModel SignatureDef contains the following input(s):
  inputs['rooms'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: feature_forward_rooms:0
  inputs['tax_rate'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: feature_forward_tax_rate:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['price'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
  outputs['tax_rate'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.